# Logistic regression

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv(r'C:\Users\Sten\Documents\EUR BAM\Thesis\data\data_cleaned_v1.csv')
df

,index,text,label
0,0,These girlfriends deserves a special mention f...,0
1,1,LeSean McCoy going through warmups with first ...,0
2,2,Tom Curran has been called up to England's Ash...,0
3,3,"We'll have turkey on the table Thursday but, a...",0
4,4,The 1945 Sinkings of the Cap Arcona and the Th...,0
...,...,...,...
499932,499995,There are a lot of things that I don't like ab...,1
499933,499996,A year after an unprecedented public outcry ag...,1
499934,499997,Battles Between the English and the Scots\n\nT...,1
499935,499998,Kurt Rambis is the new head coach of the Knick...,1


In [3]:
df = df.sample(frac=1, random_state=78735).reset_index(drop=True)
df

,index,text,label
0,46087,University atheist society president forced to...,0
1,477736,What is Ductwork in a Tub and Ductwork in a Sh...,1
2,493454,The last of the summer transfer windows is ove...,1
3,22361,"Eliot Engel, the ranking Democrat on the House...",0
4,436712,"The former CIA spy, who was the author of the ...",1
...,...,...,...
499932,459436,[There was a video here]\n\nThis week we have ...,1
499933,83541,As the war of words between the two managers s...,0
499934,140214,"It is a given fact that cats, being obligate c...",0
499935,330590,In a recent article by Chris Darden of the St....,1


In [4]:
vectorizer = TfidfVectorizer()
Tfidf = vectorizer.fit_transform(df['text'])

In [5]:
Tfidf

<499937x875483 sparse matrix of type '<class 'numpy.float64'>'
	with 106916197 stored elements in Compressed Sparse Row format>

## Define models

In [6]:
model_LR = LogisticRegression(solver='liblinear')
model_LR.fit(Tfidf, df['label'])

LogisticRegression(solver='liblinear')

In [7]:
model_SVM = LinearSVC()
model_SVM.fit(Tfidf, df['label'])

LinearSVC()

In [44]:
model_SVM2 = SVC(kernel='linear', probability=True)
model_SVM2.fit(Tfidf, df['label'])

In [ ]:
model_RF = RandomForestClassifier()
model_RF.fit(Tfidf, df['label'])

## Testing the model

In [8]:
df_test = pd.read_csv(r'C:\Users\Sten\Documents\EUR BAM\Thesis\data\data_test_v1.csv')

In [9]:
df_test = df_test.sample(frac=1, random_state=96202).reset_index(drop=True)

In [10]:
df_test

,text,label
0,A spoof of typical 1980's teen movies. The ide...,0
1,I was a victim of a false rape report by a fel...,1
2,An artist's impression of the planet Kepler 46...,1
3,During a live stream interview conducted by Dr...,0
4,"""You guys have this thing for them,"" Mr. Trump...",1
...,...,...
9995,"RICHMOND, Va. – Police responding to a report ...",1
9996,New Delhi : A day after the Narendra Modi gove...,1
9997,"""'Babysitting' scores points for sustaining a ...",0
9998,"After five years of being unemployed, a 27-yea...",1


In [11]:
Tfidf_test = vectorizer.transform(df_test['text'])

In [12]:
def scores(y, y_pred):
    accu = accuracy_score(y, y_pred)
    prec = precision_score(y, y_pred)
    rec = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)

    print('Accuracy: ', round(accu, 2))
    print('Precision: ', round(prec, 2))
    print('Recall: ', round(rec, 2))
    print("F1 Score: ", round(f1, 2))

In [13]:
predict_LR = model_LR.predict(Tfidf_test)
scores(df_test['label'], predict_LR)

Accuracy:  0.88
Precision:  0.89
Recall:  0.87
F1 Score:  0.88


In [14]:
predict_SVM = model_SVM.predict(Tfidf_test)
scores(df_test['label'], predict_SVM)

Accuracy:  0.9
Precision:  0.9
Recall:  0.9
F1 Score:  0.9


In [35]:
predict_SVM

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [15]:
model_SVM.score(Tfidf_test, df_test['label'])

0.8992

## Use model on news data

## Financial news

In [15]:
df_finance_news = pd.read_csv(r'C:\Users\Sten\Documents\EUR BAM\Thesis\data\news\finance_processed.csv')

In [16]:
Tfidf_finance = vectorizer.transform(df_finance_news['summary'])
Tfidf_finance

<2607x875483 sparse matrix of type '<class 'numpy.float64'>'
	with 161804 stored elements in Compressed Sparse Row format>

In [17]:
pred_finance = model_LR.predict_proba(Tfidf_finance)

In [18]:
pred_finance = pd.DataFrame(pred_finance)

In [19]:
pred_finance[1].mean().round(4)

0.2338

In [20]:
pred_finance.describe()

,0,1
count,2607.000000,2607.000000
mean,0.766218,0.233782
std,0.202502,0.202502
min,0.015535,0.001608
25%,0.666001,0.080888
50%,0.832699,0.167301
75%,0.919112,0.333999
max,0.998392,0.984465


## sports news

In [21]:
df_sport_news = pd.read_csv(r'C:\Users\Sten\Documents\EUR BAM\Thesis\data\news\sport_processed.csv')

In [22]:
Tfidf_sport = vectorizer.transform(df_sport_news['summary'])
Tfidf_sport

<2981x875483 sparse matrix of type '<class 'numpy.float64'>'
	with 189785 stored elements in Compressed Sparse Row format>

In [23]:
pred_sport = model_LR.predict_proba(Tfidf_sport)

In [25]:
pred_sport = pd.DataFrame(pred_sport)
pred_sport[1].mean().round(4)

0.2205

In [26]:
pred_sport.describe()

,0,1
count,2981.000000,2981.000000
mean,0.779479,0.220521
std,0.214081,0.214081
min,0.003649,0.001247
25%,0.688415,0.062890
50%,0.858466,0.141534
75%,0.937110,0.311585
max,0.998753,0.996351


## Election news

In [27]:
df_election_news = pd.read_csv(r'C:\Users\Sten\Documents\EUR BAM\Thesis\data\news\election_processed.csv')

In [28]:
Tfidf_election = vectorizer.transform(df_election_news['summary'])
Tfidf_election

<3053x875483 sparse matrix of type '<class 'numpy.float64'>'
	with 193227 stored elements in Compressed Sparse Row format>

In [29]:
pred_election = model_LR.predict_proba(Tfidf_election)

In [32]:
pred_election = pd.DataFrame(pred_election)
pred_election[1].mean().round(4)

0.2412

In [31]:
election_highprob = list(pred_election[1].sort_values(ascending=False).iloc[0:10].index)

In [26]:
highprob_news = df_election_news.loc[election_highprob, 'summary']

In [27]:
highprob_news.iloc[2]

"Labour is not gaining the seats that it would need to form a government at the next general election, a minister has said. Transport minister Richard Holden told the  BBC 's Elections 2023 programme: 'The truth is the seats that we need to win at a general election right across the country, you are just not seeing those Labour gains that they would need to do in order to form a government at the next general election at this stage.' Asked about the latest projections, Mr Holden said people have had a 'tough time' with the cost of living, adding: 'The Government has had to put a huge package in there, but people are still feeling it."

In [43]:
pred_election.describe()

,0,1
count,3053.000000,3053.000000
mean,0.758453,0.241547
std,0.204752,0.204752
min,0.014242,0.001338
25%,0.655025,0.082119
50%,0.819991,0.180009
75%,0.917881,0.344975
max,0.998662,0.985758
